In [4]:
%pip install pandas




  Using cached pandas-2.2.3-cp39-cp39-win_amd64.whl (11.6 MB)
  Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)


You should consider upgrading via the 'd:\python\traffic_signal\newenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# 1. Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])



In [ ]:
# 3. Custom Dataset
class TrafficDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.root_dir + '/' + self.data.iloc[idx]['Path']
        image = Image.open(img_path).convert('RGB')
        label = int(self.data.iloc[idx]['ClassId'])
        
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [9]:

# 4. Load train and test datasets
train_dataset = TrafficDataset(csv_file='archive/Train.csv', root_dir='archive', transform=transform)
valid_dataset = TrafficDataset(csv_file='archive/Test.csv', root_dir='archive', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [10]:
from tqdm import tqdm

# 5. Load Pretrained Model
model = models.resnet18(pretrained=True)
num_classes = len(set(train_dataset.data['ClassId']))
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 6. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)




d:\python\traffic_signal\newenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\python\traffic_signal\newenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
# 7. Training loop
num_epochs=3

for epoch in range(3):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")

# 8. Validation loop
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in valid_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

print(f"Validation Accuracy: {100*correct/total:.2f}%")

Epoch 1/3 - Training: 100%|██████████| 1226/1226 [09:26<00:00,  2.16it/s]


Epoch 1, Loss: 0.1907, Accuracy: 95.86%


Epoch 2/3 - Training: 100%|██████████| 1226/1226 [07:53<00:00,  2.59it/s]


Epoch 2, Loss: 0.0089, Accuracy: 99.82%


Epoch 3/3 - Training: 100%|██████████| 1226/1226 [06:08<00:00,  3.32it/s]


Epoch 3, Loss: 0.0077, Accuracy: 99.82%
Validation Accuracy: 99.14%


In [16]:
import torch
from torchvision import transforms
from PIL import Image

# Make sure the model is in evaluation mode
model.eval()

# Function to load and preprocess an image
def load_image(image_path):
    # Define the same transformations used during training
    transform = transforms.Compose([
        transforms.Resize((224, 224)),        # Resize to match input size
        transforms.ToTensor(),                # Convert to tensor
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
    ])
    
    image = Image.open(image_path).convert("RGB")  # Open and convert the image
    image = transform(image).unsqueeze(0)          # Add batch dimension (1,)
    return image

# Load a sample image
image_path = "left.png"  # Replace with your image path
image = load_image(image_path)

# Send the image to the same device as the model (GPU or CPU)
image = image.to(device)

# Make the prediction
with torch.no_grad():  # Turn off gradients for inference
    output = model(image)
    
# Get the predicted class
_, predicted_class = torch.max(output, 1)

# Display the predicted class
print(f"Predicted Class: {predicted_class.item()}")


Predicted Class: 34


In [17]:
# Save the model
torch.save(model.state_dict(), 'traffic_sign_model.pth')
print("Model saved!")


Model saved!


In [18]:
class_names = {
    0: "Speed limit (20km/h)",
    1: "Speed limit (30km/h)",
    2: "Speed limit (50km/h)",
    3: "Speed limit (60km/h)",
    4: "Speed limit (70km/h)",
    5: "Speed limit (80km/h)",
    6: "End of speed limit (80km/h)",
    7: "Speed limit (100km/h)",
    8: "Speed limit (120km/h)",
    9: "No passing",
    10: "No passing for vehicles over 3.5 metric tons",
    11: "Right-of-way at the next intersection",
    12: "Priority road",
    13: "Yield",
    14: "Stop",
    15: "No vehicles",
    16: "Vehicles over 3.5 metric tons prohibited",
    17: "No entry",
    18: "General caution",
    19: "Dangerous curve to the left",
    20: "Dangerous curve to the right",
    21: "Double curve",
    22: "Bumpy road",
    23: "Slippery road",
    24: "Road narrows on the right",
    25: "Road work",
    26: "Traffic signals",
    27: "Pedestrians",
    28: "Children crossing",
    29: "Bicycles crossing",
    30: "Beware of ice/snow",
    31: "Wild animals crossing",
    32: "End of all speed and passing limits",
    33: "Turn right ahead",
    34: "Turn left ahead",
    35: "Ahead only",
    36: "Go straight or right",
    37: "Go straight or left",
    38: "Keep right",
    39: "Keep left",
    40: "Roundabout mandatory",
    41: "End of no passing",
    42: "End of no passing by vehicles over 3.5 metric tons"
}


In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image

# 1. Define the model architecture (same as before)
model = models.resnet18(pretrained=False)
num_classes = 43  # replace 43 with your actual number of classes
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# 2. Load the saved weights
model.load_state_dict(torch.load('traffic_sign_model.pth', map_location='cuda' if torch.cuda.is_available() else 'cpu'))
model = model.to(device)
model.eval()
print("Model loaded!")

# 3. Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])




d:\python\traffic_signal\newenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\python\traffic_signal\newenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\HP\AppData\Local\Temp\ipykernel_22880\1182255130.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` wi

Model loaded!
Predicted Class: 14
Predicted Class: Stop


In [25]:
# 4. Load and Predict an image
def predict(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dimension

    with torch.no_grad():
        output = model(image)
        _, predicted_class = torch.max(output, 1)

    return predicted_class.item()

# Example usage
image_path = 'br.webp'  # replace this
predicted_label = predict(image_path)
print(f"Predicted Class: {predicted_label}")



class_name = class_names.get(predicted_label, "Unknown Class")
print(f"Predicted Class: {class_name}")

Predicted Class: 22
Predicted Class: Bumpy road
